### Install dependencies

In [2]:
%pip install llama-index-vector-stores-pinecone
%pip install pinecone-client>=3.0.0
%pip install "arize-phoenix[evals]" gcsfs nest-asyncio "llama-index>=0.10.3" "openinference-instrumentation-llama-index>=1.0.0" "llama-index-callbacks-arize-phoenix>=0.1.2"
%pip install llama-index-llms-anthropic
!pip install llama-index

Note: you may need to restart the kernel to use updated packages.
zsh:1: 3.0.0 not found
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import nest_asyncio
nest_asyncio.apply()

### Initialize Phoenix for tracing

In [4]:
import phoenix as px
from llama_index.core import (
    set_global_handler,
    
)
session = px.launch_app()
set_global_handler("arize_phoenix")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/usr/local/anaconda3/envs/llm-apps/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [5]:
# session.end()

### Init Pinecone and set API keys

In [6]:
from pinecone import Pinecone, ServerlessSpec
from getpass import getpass
import sys
import os
if ("PINECONE_API_KEY" not in os.environ) and ("OPENAI_API_KEY" not in os.environ) and ("ANTHROPIC_API_KEY" not in os.environ):
  os.environ["PINECONE_API_KEY"] = getpass("Please enter your Pinecone API key: ")
  os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API key: ")
  os.environ["ANTHROPIC_API_KEY"] = getpass("Please enter your Anthropic API key: ")


api_key = os.environ["PINECONE_API_KEY"]
pc = Pinecone(api_key=api_key)

In [35]:
from llama_index.core import Settings
from llama_index.llms.anthropic import Anthropic
from llama_index.llms.openai import OpenAI

# llm = Anthropic(model="claude-3-haiku-20240307", temperature = 0)
llm = Anthropic(model="claude-3-sonnet-20240229", temperature = 0)
# llm = OpenAI(model="gpt-3.5-turbo", temperature = 0, max_tokens=240)
Settings.llm = llm


In [36]:
mood_feeling_index = pc.Index("moonsync-index-mood-feeling")
general_index = pc.Index("moonsync-index-general")
diet_nutrition_index = pc.Index("moonsync-index-diet-nutrition")
fitness_wellness_index = pc.Index("moonsync-index-fitness-wellness")
indexes = [mood_feeling_index, general_index, diet_nutrition_index, fitness_wellness_index]

In [37]:
print(mood_feeling_index.describe_index_stats())
print(general_index.describe_index_stats())
print(diet_nutrition_index.describe_index_stats())
print(fitness_wellness_index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 385}},
 'total_vector_count': 385}
{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 308}},
 'total_vector_count': 308}
{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 873}},
 'total_vector_count': 873}
{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1656}},
 'total_vector_count': 1656}


In [38]:
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.pinecone import PineconeVectorStore


vector_indexes = []
for index in indexes:
  vector_indexes.append(VectorStoreIndex.from_vector_store(PineconeVectorStore(pinecone_index=index)))

In [39]:
query_engines = []
for vector_index in vector_indexes:
  query_engines.append(vector_index.as_query_engine())
  

mood_feeling_query_engine, general_query_engine, diet_nutrition_query_engine, fitness_wellness_query_engine = query_engines

### Create tools for each category

In [40]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata


mood_feeling_tool = QueryEngineTool(
    query_engine=mood_feeling_query_engine,
    metadata=ToolMetadata(
        name="mood/feeling",
        description="Useful for questions related to mood and feelings ",
    ),
)

diet_nutrition_tool = QueryEngineTool(
    query_engine=diet_nutrition_query_engine,
    metadata=ToolMetadata(
        name="diet/nutrition",
        description="Useful for questions related to women's diet and nutrition recommendatations",
    ),
)

general_tool = QueryEngineTool(
    query_engine=general_query_engine,
    metadata=ToolMetadata(
        name="general",
        description="Useful for general questions related to women's menstrual cycle"
    ),
)

fitness_wellness_tool = QueryEngineTool(
    query_engine=fitness_wellness_query_engine,
     metadata=ToolMetadata(
        name="general",
        description="Useful for questions related to fitness and wellness advice for women"
    ),
)

In [41]:
from llama_index.core.query_engine import RouterQueryEngine, SubQuestionQueryEngine
from llama_index.core.selectors import LLMSingleSelector, LLMMultiSelector
from llama_index.core.selectors import (
    PydanticMultiSelector,
    PydanticSingleSelector,
)

#! Pydantic works only with OpenAI models
# router_query_engine = RouterQueryEngine(
#     selector=PydanticMultiSelector.from_defaults(),
#     query_engine_tools=[
#         mood_feeling_tool,
#         diet_nutrition_tool,
#         general_tool,
#         fitness_wellness_tool,
#     ],
# )

router_query_engine = RouterQueryEngine(
    selector=LLMMultiSelector.from_defaults(),
    query_engine_tools=[
        mood_feeling_tool,
        diet_nutrition_tool,
        general_tool,
        fitness_wellness_tool,
    ],
)

sub_question_query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=[
        mood_feeling_tool,
        diet_nutrition_tool,
        general_tool,
        fitness_wellness_tool,
    ],
    use_async = True
)

In [42]:
response = sub_question_query_engine.query("What are some diet recommendations during the PMS phase and how will it uplift my mood?")
print(str(response))

Generated 3 sub questions.
[diet/nutrition] Q: What are some diet recommendations for women during the PMS phase?
[mood/feeling] Q: How can diet impact mood during the PMS phase?
[diet/nutrition] Q: What foods or nutrients can help uplift mood during the PMS phase?
[mood/feeling] A: Diet can significantly impact mood during the PMS phase. Consuming whole grains, sweet potatoes, and other healthy carbohydrates can help boost serotonin production and promote a calmer mood. On the other hand, refined carbs and sugary foods like cookies can deplete nutrients and lead to mood swings. Incorporating healthy fats like olive oil, avocados, and fatty fish can also be beneficial. It's recommended to limit dairy, caffeine, salt, and refined carbs, as these can exacerbate PMS symptoms. Dark chocolate (62% or higher cacao) is an exception and can improve mood due to its ability to support neurotransmitter production. Adequate protein and healthy fats at each meal can help stabilize blood sugar level

In [43]:
response = router_query_engine.query("What are some diet recommendations during the PMS phase and how will it uplift my mood?")
print(str(response))

Here are some dietary recommendations that can help uplift your mood during the PMS phase:

1. Increase your intake of foods rich in B vitamins, such as whole grains, legumes, leafy greens, and fortified cereals. B vitamins support neurotransmitter production and can promote a calmer mood.

2. Consume foods high in omega-3 fatty acids like salmon, chia seeds, and walnuts. Omega-3s have anti-inflammatory properties that can reduce period pain and improve mood.

3. Eat magnesium-rich foods such as hemp hearts, leafy greens, nuts, and seeds. Magnesium can help alleviate muscle cramps, twitching, and mood swings associated with PMS.

4. Include iron-rich foods like lean red meat, lentils, and spinach in your diet. Iron deficiency can worsen PMS symptoms like fatigue, mood swings, and brain fog.

5. Incorporate zinc-rich foods like oysters, beef, lamb, spinach, cashews, and pumpkin seeds. Zinc plays a role in hormone metabolism and may reduce PMS symptoms.

6. Stay hydrated by drinking plen